In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pyuvdata import UVData, UVCal, UVFlag, utils
from glob import glob
from astropy import units as u
from astropy import constants as c
from astropy.time import Time
from astropy.coordinates import get_body, SkyCoord, AltAz, EarthLocation, Angle
from copy import deepcopy

In [1]:
JD = '2457548'
datadir = '/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457548/'

In [3]:
ls /lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457548/

2457548_summary.csv
lst.2457548.10h36m00s.10h42m00s.uvcRP.drift.uvh5
lst.2457548.10h42m00s.10h48m00s.uvcRP.drift.uvh5
lst.2457548.10h48m00s.10h54m00s.uvcRP.drift.uvh5
lst.2457548.10h54m00s.11h00m00s.uvcRP.drift.uvh5
lst.2457548.11h00m00s.11h06m00s.uvcRP.drift.uvh5
lst.2457548.11h06m00s.11h12m00s.uvcRP.drift.uvh5
lst.2457548.11h12m00s.11h18m00s.uvcRP.drift.uvh5
lst.2457548.11h18m00s.11h24m00s.uvcRP.drift.uvh5
lst.2457548.11h24m00s.11h30m00s.uvcRP.drift.uvh5
lst.2457548.11h30m00s.11h36m00s.uvcRP.drift.uvh5
lst.2457548.11h36m00s.11h42m00s.uvcRP.drift.uvh5
lst.2457548.11h42m00s.11h48m00s.uvcRP.drift.uvh5
lst.2457548.11h48m00s.11h54m00s.uvcRP.drift.uvh5
lst.2457548.11h54m00s.12h00m00s.uvcRP.drift.uvh5
lst.2457548.12h00m00s.12h06m00s.uvcRP.drift.uvh5
lst.2457548.12h06m00s.12h12m00s.uvcRP.drift.uvh5
lst.2457548.12h12m00s.12h18m00s.uvcRP.drift.uvh5
lst.2457548.12h18m00s.12h24m00s.uvcRP.drift.uvh5
lst.2457548.12h24m00s.12h30m00s.uvcRP.drift.uvh5
lst.2457548.12h30m00s.12h36m00s.uvcRP.drift.uvh5


In [4]:
files = glob(datadir+'zen.'+JD+'**'+'uvcRP.uvh5')

In [6]:
#loop to make flags 

In [ ]:
for file in files 
    #prep data before making different types of flags 
    uvd = UVData()
    uvd.read(file)
    uvf = UVFlag(uvd, mode='flag', copy_flags=True, run_check=True)
    uvf_or = deepcopy(uvf)
    uvf_thresh = deepcopy(uvf)
    uvf_greedy = deepcopy(uvf)
    
    #uvf_or
    uvf_or.to_waterfall(method='or', keep_pol=False)
    #uvf_thresh
    uvf_thresh.to_waterfall(method='mean', keep_pol=False)
    #uvf_greedy (not sure if this is right)
    uvf_greedy.to_waterfall(method='or', keep_pol=False)
    uvf_greedy.flag_array[time_frac>0.33,:,0] = True
    uvf_greedy.flag_array[:,freq_frac>0.10,0] = True
    
    #do i need to read out the uvflag data? not sure how to name the files
    #uvf_or.write('flag......or.uvh5', clobber=True)
    #uvf_thresh.write('flag......or.uvh5', clobber=True)
    #uvf_greedy.write('flag.......or.uvh5', clobber=True)
    
    #applying the flags to the data...i think i need to make copies of the uvd..?
    uvd_or = deepcopy(uvd)
    uvd_thresh = deepcopy(uvd)
    uvd_greedy = deepcopy(uvd)
    utils.apply_uvflag(uvd_or, uvf_or)
    utils.apply_uvflag(uvd_thresh, uvf_thresh)
    utils.apply_uvflag(uvd_greedy, uvf_greedy)
    
    #write out the uvd (also not sure about naming the files)
    #uvd_or.write()
    #uvd_thresh.write()
    #uvd_greedy.write()